### Create a product that builds a Brochure for a recruitement company in London to be used for prospective clients, investors and potential recruits.

In [4]:
# import packages
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display


In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
""" A class that represents a web page and provides utilities for retrieving its content and embedded links.
    This class fetches the content of a given webpage URL, extracts and stores the page's HTML body,
    title, and text content while omitting certain non-essential elements (e.g., scripts and styles).
    It also parses and collects all hyperlinks found on the page.
"""


class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')] # retutn the links
        self.links = [link for link in links if link]  # this is return a us list of links 

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://www.high-society.co.uk/")
print(ed.get_contents())

Webpage Title:
High Society: Best Catering Staff in London for Your Event
Webpage Contents:
top of page
Find staff
Find work
Meet the team
News
Talk to us
Staff portal
Hand-selected staff for every event
That’s what we offer to every one of our clients – carefully trained, highly articulate and always immaculate individ
uals and teams for events of all kinds, at some of the very best venues in London.
Join the best team
Find the right staff
07795 425407
The highest standard of staff
For over 20 years, we’ve been hiring and providing only
the most personable, presentable and reliable event staff in London.
And we’ll never stop looking for more and training them better. So what can we do for you?
Tell us what you need
Work that suits you, at venues you’ll love
We want to deliver
the best events
for our clients – and to do that, we need
the best people.
So if that’s you, we need you – for work that fits around your life, with a team of people like you and experiencing some incredible venu

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
I will use "one shot prompting" in which we provide an example of how it should respond in the prompt.
This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!
Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec.

In [12]:
# here I specify a system promopt 
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""   

In [13]:
#this where I define user prompt
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [14]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.high-society.co.uk/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.high-society.co.uk
https://www.high-society.co.uk/find-staff
https://www.high-society.co.uk/find-work
https://www.high-society.co.uk/our-difference
https://www.high-society.co.uk/news
https://www.high-society.co.uk/contact
https://staff.high-society.co.uk/LoginPage?ReturnUrl=%2F
https://high-society-staff.co.uk/application
https://www.high-society.co.uk/find-staff
https://www.high-society.co.uk/contact
https://high-society-staff.co.uk/application
mailto:info@high-society.co.uk?subject=Enquiry 
mailto:info@high-society.co.uk?subject=Enquiry%20from%20the%20website
tel:+44 (0)20 7228 0333
tel:+44 (0)20 7228 0333
tel:+44 (0)20 7228 0333
https://www.instagram.com/highsocietystaff

In [15]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}  # this where I say to chatgpt to retun a as json object  it needs to in prompt and in response prompt in both of them 
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [18]:
get_links("https://www.high-society.co.uk")

{'links': [{'type': 'homepage', 'url': 'https://www.high-society.co.uk'},
  {'type': 'find staff page',
   'url': 'https://www.high-society.co.uk/find-staff'},
  {'type': 'find work page',
   'url': 'https://www.high-society.co.uk/find-work'},
  {'type': 'our difference page',
   'url': 'https://www.high-society.co.uk/our-difference'},
  {'type': 'news page', 'url': 'https://www.high-society.co.uk/news'},
  {'type': 'contact page', 'url': 'https://www.high-society.co.uk/contact'}]}

## Second step: make the brochure!


In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://www.high-society.co.uk"))

Found links: {'links': [{'type': 'home page', 'url': 'https://www.high-society.co.uk'}, {'type': 'find staff page', 'url': 'https://www.high-society.co.uk/find-staff'}, {'type': 'find work page', 'url': 'https://www.high-society.co.uk/find-work'}, {'type': 'our difference page', 'url': 'https://www.high-society.co.uk/our-difference'}, {'type': 'news page', 'url': 'https://www.high-society.co.uk/news'}, {'type': 'contact page', 'url': 'https://www.high-society.co.uk/contact'}]}
Landing page:
Webpage Title:
High Society: Best Catering Staff in London for Your Event
Webpage Contents:
top of page
Find staff
Find work
Meet the team
News
Talk to us
Staff portal
Hand-selected staff for every event
That’s what we offer to every one of our clients – carefully trained, highly articulate and always immaculate individ
uals and teams for events of all kinds, at some of the very best venues in London.
Join the best team
Find the right staff
07795 425407
The highest standard of staff
For over 20 year

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result)) # this help me to gov enice style to the page

In [26]:
create_brochure("High society", "https://www.high-society.co.uk")  # the logic of this is first send a request where find the most usefelf link and then send back the resposne to chat gopt and tell to crete a brocher form that new information 


Found links: {'links': [{'type': 'about page', 'url': 'https://www.high-society.co.uk/our-difference'}, {'type': 'careers page', 'url': 'https://www.high-society.co.uk/find-work'}, {'type': 'find staff page', 'url': 'https://www.high-society.co.uk/find-staff'}, {'type': 'contact page', 'url': 'https://www.high-society.co.uk/contact'}, {'type': 'news page', 'url': 'https://www.high-society.co.uk/news'}]}


# High Society Brochure

---

## About Us

Welcome to **High Society**, London’s premier catering staff agency with over **20 years** of excellence in the hospitality industry. Since 1998, we have been dedicated to hand-selecting and training the most personable, presentable, and reliable staff for events of all kinds. Our goal is to ensure every event is memorable and runs seamlessly.

## Our Services

At High Society, we provide **carefully trained teams** for a variety of roles including:
- **Team Players**: Deliver exceptional service, from dinner service to drink serving.
- **Ambassadors**: Coordinate team arrivals and maintain high standards of performance.
- **Bar Staff**: Provide a smooth drink service with charm and efficiency.
- **Team Leaders**: Oversee teams and ensure compliance with dietary requirements.
- **Baristas**: Craft specialty drinks with skill and care.

Tell us what you need, and we’ll provide exactly the right staff to ensure your special event shines!

## Company Culture

We believe in a warm and inviting workplace, where flexibility and fun are key. Our team reflects our commitment to excellence, and as a member of our staff, you can expect:
- **Flexibility**: Work when and where fits your schedule.
- **Friendship**: Collaborate with like-minded individuals and create lasting connections.
- **Growth**: Training and opportunities to enhance your skills and advance in your career.

## Our Clients

We cater to a wide range of venues and events, including:
- High-profile corporate events
- Lavish weddings
- Intimate private parties
- Fundraising galas

Our clients praise our staff for their professionalism, adaptability, and friendly demeanor, allowing us to cultivate long-term relationships built on trust and satisfaction.

## Giving Back

High Society is proud to support **Great Ormond Street Hospital**, providing complimentary staff for fundraisers and initiatives that aim to improve the lives of sick children and their families. We believe in making a difference in our community, one event at a time.

## Careers and Opportunities

Interested in joining our dynamic team? We’re looking for:
- **Specialists** in hospitality who share our values of warmth, professionalism, and positivity.
- **Individuals** who are ready to take on flexible roles at some of London’s most exciting venues.

**Apply here** to become part of London’s best catering staff today!

## Contact Us

Have questions or want to discuss staffing your next event? **Talk to us!**
- ✉️ Email: [info@high-society.co.uk](mailto:info@high-society.co.uk)
- 📞 Phone: 07795 425407 / 020 7228 0333
- 📸 Follow us on Instagram [@highsocietystaff](https://instagram.com/highsocietystaff)

---

Join us at High Society and help create unforgettable events with a team that embodies excellence!

## The familiar typewriter animation

this is minor improvement which help me insted of  wait the data the promtpo to come complete the response is stream this thanks to stream = True

In [30]:
def stream_brochure(company_name, url): +
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True  
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("bee recruit", "https://beerecruit.co.uk/")

Found links: {'links': [{'type': 'home page', 'url': 'https://beerecruit.co.uk/'}, {'type': 'about page', 'url': 'https://beerecruit.co.uk/ethos/'}, {'type': 'team page', 'url': 'https://beerecruit.co.uk/team/'}, {'type': 'jobs page', 'url': 'https://beerecruit.co.uk/jobs/'}, {'type': 'candidates page', 'url': 'https://beerecruit.co.uk/candidates/'}, {'type': 'clients page', 'url': 'https://beerecruit.co.uk/clients/'}, {'type': 'partners page', 'url': 'https://beerecruit.co.uk/partners/'}, {'type': 'consultancy page', 'url': 'https://beerecruit.co.uk/consultancy/'}]}



# Bee Recruitment Brochure

Welcome to **Bee Recruitment**, your trusted partner in hospitality recruitment. As a premier agency based in London, we bring nearly **20 years of experience** to the table, connecting skilled hospitality professionals with leading establishments. 

## Our Ethos

At Bee Recruitment, we pride ourselves on a straightforward approach: **We listen carefully, learn quickly, and commit to making people happy**. We believe in:

- **People First**: The heart of our operations, placing individuals at the forefront.
- **Guidance & Support**: Providing expert advice derived from years of experience.
- **Honesty**: Engaging in transparent communication without any nonsense.
- **Commitment**: Dedicating ourselves fully to the success of our candidates and clients.

## Our Expertise

Navigating the complex world of hospitality recruitment can be daunting. That's why our team, made up of seasoned hospitality experts, is here to help you find the right fit—whether you’re a candidate seeking new opportunities or a business looking to recruit top talent. 

### Industries We Serve:
- Hotels
- Restaurants
- Catering Companies
- Private Houses

## Meet the Team

Our Managing Director, **Rory Laffan**, along with our talented team of professionals—including **James Watson** (Operations Director), **Adam Probets** (Director – South West), and **Scott Bradley** (Head of Recruitment – The North)—are all deeply rooted in the hospitality industry. Each brings unique expertise and a commitment to fostering long-term relationships built on trust and respect.

## Our Clients

Over the years, Bee Recruitment has forged partnerships with an array of distinguished clients across the UK and beyond, including prestigious restaurants, luxury hotels, and private dining establishments. 

Our network extends beyond London, encompassing Bristol and various regions across Europe, the Americas, the Middle East, and the Caribbean.

## Career Opportunities

We currently offer a wide array of exciting positions in hospitality, such as:

- General Manager
- Assistant General Manager
- Head Chef
- Sales & Revenue Director
- Sous Chef
- Front of House Roles

If you’re looking for your next career move, send us your CV and let’s discuss how we can help you achieve your goals!

## Giving Back

At Bee Recruitment, we believe in making a positive impact within our community. Supporting meaningful charities is an integral part of our company values, and we invite you to discover the remarkable organizations we partner with.

## Why Choose Bee?

Choosing Bee means you’re not just selecting a recruitment agency; you're selecting a dedicated partner deeply enmeshed in the hospitality community. Our passion fuels our commitment to helping individuals and businesses thrive.

### Contact Us

Interested in working with us? Great! 
- **Address**: York House, 221 Pentonville Rd, London N1 9UZ, UK
- **Email**: [email protected]
- **Follow Us**: [Instagram](https://instagram.com/bee_hospitality) | [Facebook](https://facebook.com) | [LinkedIn](https://linkedin.com)

Bee Recruitment—where expertise meets passion in hospitality recruitment.



## Business Applications
In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.